## LLM

In [1]:
import sys
sys.path.insert(0, '../src/')

import matplotlib.pyplot as plt
from tokenizer import TokenizerBPE
from data_handling import normalize_to_ascii, clean_text
from utils import saver, loader
import numpy as np
import re

import os
import pickle as pkl
from tqdm.notebook import tqdm
import json

# disable gpu for testing purposes
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


## General Math

In [2]:
with open('../corpus/train.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

length = len(data)

In [3]:
print(length)

7473


In [4]:
question_list = []
answer_list = []


for i, sample in tqdm(enumerate(data), total=length):
    question = sample["question"]
    question = clean_text(normalize_to_ascii(question))
    answer = clean_text(normalize_to_ascii(sample["answer"]))
    answer = re.sub(r"<<.*?>>", "", answer)
    answer = re.sub(r"####.*", "", answer).strip()
    question_list.append(question)
    answer_list.append(answer)

  0%|          | 0/7473 [00:00<?, ?it/s]

In [5]:
print(len("".join(question_list + answer_list)))

3533543


In [6]:
saver("../corpus/math_raw", [question_list, answer_list])

In [8]:
def fused_sqa(story_list, question_grouped_list, answer_grouped_list):
    q ="<q>"
    a = "<a>"
    sos = "<s>"
    eos = "</s>"

    rcw = re.compile(r"\s+")

    corpus_list = []
    for story, question_list, answer_list in tqdm(list(zip(story_list, question_grouped_list, answer_grouped_list))):
        sqa = [sos, normalize_to_ascii(story).lower()]
        for question, answer in zip(question_list, answer_list):
            sqa.append(q)
            sqa.append(normalize_to_ascii(question).lower())
            sqa.append(a)
            sqa.append(normalize_to_ascii(answer).lower())
        sqa.append(eos)
        corpus_list.append("".join(sqa))
        
    return corpus_list

In [11]:
corpus_list = fused_sqa(context_list, question_grouped, answer_grouped)

  0%|          | 0/7199 [00:00<?, ?it/s]

## 